In [1]:
import os
import pandas as pd

# 'data/raw_data' 경로 설정
root_dir = 'data/'
visitor_city = pd.read_csv(os.path.join(root_dir, "city_of_festival_visitors.csv"))
address = pd.read_csv(os.path.join(root_dir, "address_with_lon_lat_final.csv"))
mooju = set(list(address[address['시군구명'] == '무주군']['행정동코드']))
other_city = list(address.merge(visitor_city, on=['시도명', '시군구명'])['행정동코드_x'])

In [2]:
import os
import pandas as pd

od_df = pd.DataFrame()
raw_data_dir = os.path.join(root_dir, "raw_data/")
output_file_path = os.path.join(root_dir, "od_df.csv")  # od_df.csv 파일 경로

# 전처리코드는 모두 여기에
def preprocess(df):
    # dest_hdong_cd 값이 축제가 열리는 곳인 무주군 데이터만 필터링
    #filtered_df = df[df['dest_hdong_cd'].isin(mooju)]

    # origin_hdong_cd 값이 visitor_city 인 데이터만 필터링
    #filtered_df = filtered_df[filtered_df['origin_hdong_cd'].isin(other_city)]

    """# 타 지역에서 온 데이터만 필터링
    filtered_df = filtered_df[~filtered_df['origin_hdong_cd'].isin(mooju)]"""
    filtered_df = df[df['modal'] == 0]
    return filtered_df

# od_df.csv 파일이 존재하는지 확인
if os.path.exists(output_file_path):
    print("od_df.csv 파일이 존재하여 불러옵니다.")
    od_df = pd.read_csv(output_file_path)
else:
    print("od_df.csv 파일이 존재하지 않아 새로운 od_df 데이터프레임을 생성합니다.")
    # 'od'로 시작하는 폴더 내의 모든 CSV 파일 처리
    for dirpath, dirnames, filenames in os.walk(raw_data_dir):
        if os.path.basename(dirpath).startswith('od'):
            for filename in filenames:
                if filename.endswith('.csv'):
                    # 파일 이름에서 날짜 추출
                    date_str = filename.split('_')[1]

                    # 파일 경로 설정 및 CSV 읽기
                    file_path = os.path.join(dirpath, filename)
                    csv_data = pd.read_csv(file_path)

                    # 전처리
                    filtered_data = preprocess(csv_data)

                    # 날짜에서 월일(MMDD) 부분 추출
                    mmdd_str = date_str[4:]  # 'YYYYMMDD'에서 마지막 네 자리 'MMDD' 추출

                    # 동적으로 변수 생성 (예: df_0901)
                    globals()[f'df_{mmdd_str}'] = filtered_data

                    # 데이터프레임 이름 출력 (예: df_0901)
                    print(f"데이터프레임 df_{mmdd_str}가 생성되었고, 기존 데이터프레임에 포함합니다.")
                    od_df = pd.concat([od_df, globals()[f'df_{mmdd_str}']])

        # 데이터 처리 완료 후 od_df.csv 파일로 저장
        od_df.to_csv(output_file_path, index=False)
        print(f"데이터프레임 od_df가 '{output_file_path}'에 저장되었습니다.")


In [3]:
visitor_city['시도 시군구'] = visitor_city['시도명'].fillna('') + ' ' + visitor_city['시군구명'].fillna('')

# 양쪽 값이 모두 null인 경우 빈 문자열 처리
visitor_city['시도 시군구'] = visitor_city['시도 시군구'].str.strip()
visitor_city.head()

,시군구명,od_cnts,시도명,행정동코드,위도,경도,시도 시군구
0,전주시 덕진구,2834,전라북도,4511300000,35.847561,127.117672,전라북도 전주시 덕진구
1,서구,1813,대전광역시,3017000000,36.355179,127.383849,대전광역시 서구
2,전주시 완산구,1561,전라북도,4511100000,35.795512,127.132447,전라북도 전주시 완산구
3,유성구,1379,대전광역시,3020000000,36.362073,127.356410,대전광역시 유성구
4,동구,1207,대전광역시,3011000000,35.886500,128.635500,대전광역시 동구


In [4]:
address['시도 시군구'] = address['시도명'].fillna('') + ' ' + address['시군구명'].fillna('')

# 양쪽 값이 모두 null인 경우 빈 문자열 처리
address['시도 시군구'] = address['시도 시군구'].str.strip()
address.head()

,행정동코드,시도명,시군구명,읍면동명,동리명,위도,경도,시도 시군구
0,1100000000,서울특별시,NaN,NaN,서울특별시,37.566679,126.978291,서울특별시
1,1111000000,서울특별시,종로구,NaN,종로구,37.580695,126.982799,서울특별시 종로구
2,1111051500,서울특별시,종로구,청운효자동,세종로,37.579997,126.976930,서울특별시 종로구
3,1111051500,서울특별시,종로구,청운효자동,옥인동,37.583480,126.963850,서울특별시 종로구
4,1111051500,서울특별시,종로구,청운효자동,누하동,37.578998,126.967561,서울특별시 종로구


### 충남(세종/대전)지역 최적의 셔틀 노선

In [5]:
daejun_shuttle = ['세종특별자치시', '대전광역시 유성구', '대전광역시 서구', '대전광역시 대덕구', '대전광역시 중구', '충청남도 금산군', '충청북도 영동군', '전라북도 무주군']

In [6]:
duration_table_dj = []
distance_table_dj = []

for idx,target in enumerate(daejun_shuttle):
    origin_criteria = list(set(address[address['시도 시군구'] == target]['행정동코드']))
    
    duration_arr = []
    distance_arr = []
    
    for idy, other in enumerate(daejun_shuttle):
        if idx == idy:
            od_duration = od_dist = 0
        else:
            dest_criteria = list(set(address[address['시도 시군구'] == other]['행정동코드']))
            tmp_df = od_df[
                od_df['origin_hdong_cd'].isin(origin_criteria) & od_df['dest_hdong_cd'].isin(dest_criteria)
            ]
            od_duration = tmp_df['od_duration_avg'].mean()
            od_dist = tmp_df['od_dist_avg'].mean()
        duration_arr.append(od_duration)
        distance_arr.append(od_dist)

    duration_table_dj.append(duration_arr)
    distance_table_dj.append(distance_arr)

In [7]:
for i in range(len(duration_table_dj)):
    for j in duration_table_dj[i]:
        print(f"{j:.2f}", end="     ")
    print() 

0.00     46.17     70.92     65.31     80.85     100.55     115.87     136.59     
62.91     0.00     35.83     42.19     56.48     80.77     100.71     114.20     
79.69     30.86     0.00     41.36     33.97     76.64     100.27     109.01     
70.42     36.54     48.28     0.00     48.55     78.41     87.53     111.99     
90.38     56.31     34.15     41.00     0.00     65.92     96.19     103.16     
100.54     76.89     75.28     75.00     60.59     0.00     61.08     59.22     
109.96     98.58     100.00     84.42     94.59     64.07     0.00     53.32     
132.54     103.30     108.36     101.98     97.18     64.16     52.17     0.00     


In [8]:
for i in range(len(distance_table_dj)):
    for j in distance_table_dj[i]:
        print(f"{j:.2f}", end="     ")
    print()

0.00     38633.36     62825.13     66274.91     76947.05     122255.86     145235.23     183927.22     
59529.10     0.00     20222.39     25852.58     36679.50     90046.29     119844.34     157934.80     
76722.15     16139.82     0.00     22061.65     18424.38     76846.87     114151.80     151477.61     
77512.97     22307.94     29841.62     0.00     31219.22     85099.29     99159.12     152123.38     
95283.47     33367.76     18341.32     23686.71     0.00     60852.27     104719.50     129015.88     
130339.75     86420.69     74127.01     81013.54     55402.35     0.00     64867.52     71600.40     
155409.07     115832.26     112761.76     101692.48     106304.84     67943.59     0.00     56135.20     
200733.67     145413.37     143672.53     147402.42     133664.60     85216.90     56097.19     0.00     


In [9]:
import numpy as np
import networkx as nx

# 정거장 후보지와 가중치
stations = {
    "세종특별자치시": 1,
    "대전광역시 유성구": 2,  # 가중치 2로 증가
    "대전광역시 서구": 2,    # 가중치 2로 증가
    "대전광역시 대덕구": 1,
    "대전광역시 중구": 1,
    "충청남도 금산군": 1,
    "충청북도 영동군": 2,    # 가중치 2로 증가
    "전라북도 무주군": 0,    # 목적지
}

# 그래프 생성
G = nx.DiGraph()

# 그래프에 엣지 추가
for i, from_station in enumerate(stations.keys()):
    for j, to_station in enumerate(stations.keys()):
        if i != j:
            G.add_edge(from_station, to_station, 
                       distance=distance_table_dj[i][j], 
                       time=duration_table_dj[i][j])

# 모든 경로 찾기
def find_all_routes(G, start, target, visited=None, path=None):
    if visited is None:
        visited = set()
    if path is None:
        path = []

    visited.add(start)
    path.append(start)

    if start == target:
        yield path.copy()
    else:
        for neighbor in G.neighbors(start):
            if neighbor not in visited:
                yield from find_all_routes(G, neighbor, target, visited, path)

    path.pop()
    visited.remove(start)

# 최적 경로 탐색 함수
def find_optimal_routes_within_range(G, stations, min_factor, max_factor, step):
    optimal_routes = []
    max_stops = 0
    explored_routes = set()  # 탐색된 경로를 저장할 집합

    # 여러 배수 값을 탐색
    for factor in np.arange(min_factor, max_factor + step, step):

        for station in stations.keys():
            if station != "전라북도 무주군":  # 목적지가 아니면
                routes = list(find_all_routes(G, station, "전라북도 무주군"))

                for route in routes:
                    valid_route = True
                    total_distance = 0
                    total_time = 0

                    # 각 경로의 모든 구간에 대해 확인
                    for i in range(len(route) - 1):
                        current_station = route[i]

                        # 현재 노드에서 무주군으로 바로 이동할 때의 거리와 시간
                        if "전라북도 무주군" in G[current_station]:  # 직접 경로가 있는 경우에만
                            direct_distance = G[current_station]["전라북도 무주군"]['distance']
                            direct_time = G[current_station]["전라북도 무주군"]['time']

                            # 현재 구간의 거리와 시간을 누적
                            total_distance += G[route[i]][route[i + 1]]['distance']
                            total_time += G[route[i]][route[i + 1]]['time']

                            # 각 중간 노드에서도 주어진 배수(factor) 조건 확인
                            if total_distance > direct_distance * factor or total_time > direct_time * factor:
                                valid_route = False
                                break

                    if valid_route:
                        # 경로를 집합에 저장할 수 있는 형태로 변환 (정렬하여 순서 상관없이 비교 가능)
                        route_tuple = tuple(sorted(route))

                        # 만약 해당 경로가 이전 배수에서 이미 탐색되었다면 제외
                        if route_tuple not in explored_routes:
                            explored_routes.add(route_tuple)  # 새로운 경로로 저장
                            # 가장 많은 정거장 수 확인
                            if len(route) > max_stops:
                                max_stops = len(route)
                                optimal_routes = [(route, total_distance, total_time, factor)]
                            elif len(route) == max_stops:
                                optimal_routes.append((route, total_distance, total_time, factor))

    return optimal_routes

# 탐색 범위 설정 (예: 1.1부터 1.5까지 0.1 간격으로 탐색)
min_factor = 1.1
max_factor = 1.5
step = 0.1

# 최적 경로 탐색 실행
optimal_routes = find_optimal_routes_within_range(G, stations, min_factor, max_factor, step)

# 결과 출력
print("충남(세종/대전)지역 최적의 셔틀 노선")
if optimal_routes:
    for route, total_distance, total_time, factor in optimal_routes:
        print(f"최적의 셔틀 노선: {route} | 총 거리: {total_distance} | 총 시간: {total_time} | 배수: {factor:.2f}")
else:
    print("유효한 경로가 없습니다.")


충남(세종/대전)지역 최적의 셔틀 노선
최적의 셔틀 노선: ['대전광역시 서구', '대전광역시 유성구', '전라북도 무주군'] | 총 거리: 174074.62193568642 | 총 시간: 145.06581350362478 | 배수: 1.30
최적의 셔틀 노선: ['대전광역시 유성구', '대전광역시 대덕구', '전라북도 무주군'] | 총 거리: 177975.96865794668 | 총 시간: 154.17799808540127 | 배수: 1.40
최적의 셔틀 노선: ['대전광역시 서구', '대전광역시 대덕구', '전라북도 무주군'] | 총 거리: 174185.03447148675 | 총 시간: 153.34457432947366 | 배수: 1.40
최적의 셔틀 노선: ['대전광역시 서구', '대전광역시 중구', '전라북도 무주군'] | 총 거리: 147440.26375315106 | 총 시간: 137.13571595994017 | 배수: 1.40
최적의 셔틀 노선: ['대전광역시 중구', '대전광역시 대덕구', '전라북도 무주군'] | 총 거리: 175810.09228782012 | 총 시간: 152.99119343461874 | 배수: 1.40
최적의 셔틀 노선: ['세종특별자치시', '대전광역시 유성구', '전라북도 무주군'] | 총 거리: 196568.16680068919 | 총 시간: 160.37963214673238 | 배수: 1.50
최적의 셔틀 노선: ['대전광역시 중구', '대전광역시 유성구', '전라북도 무주군'] | 총 거리: 191302.567082131 | 총 시간: 170.5166103299462 | 배수: 1.50


### 전북지역 최적의 셔틀 노선

In [10]:
jeonbuk_shuttle = ['전라북도 군산시', '전라북도 익산시', '전라북도 전주시 완산구', '전라북도 전주시 덕진구', '전라북도 진안군', '전라북도 장수군', '전라북도 무주군']

In [11]:
duration_table_jb = []
distance_table_jb = []

for idx,target in enumerate(jeonbuk_shuttle):
    origin_criteria = list(set(address[address['시도 시군구'] == target]['행정동코드']))
    
    duration_arr = []
    distance_arr = []
    
    for idy, other in enumerate(jeonbuk_shuttle):
        if idx == idy:
            od_duration = od_dist = 0
        else:
            dest_criteria = list(set(address[address['시도 시군구'] == other]['행정동코드']))
            tmp_df = od_df[
                od_df['origin_hdong_cd'].isin(origin_criteria) & od_df['dest_hdong_cd'].isin(dest_criteria)
            ]
            od_duration = tmp_df['od_duration_avg'].mean()
            od_dist = tmp_df['od_dist_avg'].mean()
        duration_arr.append(od_duration)
        distance_arr.append(od_dist)

    duration_table_jb.append(duration_arr)
    distance_table_jb.append(distance_arr)

In [12]:
import numpy as np
import networkx as nx

# 정거장 후보지와 가중치
stations = {
    '전라북도 군산시': 1, 
    '전라북도 익산시': 1, 
    '전라북도 전주시 완산구': 2, 
    '전라북도 전주시 덕진구': 2, 
    '전라북도 진안군': 1, 
    '전라북도 장수군': 2,
    '전라북도 무주군': 0,    # 목적지
}

# 그래프 생성
G = nx.DiGraph()

# 그래프에 엣지 추가
for i, from_station in enumerate(stations.keys()):
    for j, to_station in enumerate(stations.keys()):
        if i != j:
            G.add_edge(from_station, to_station, 
                       distance=distance_table_jb[i][j], 
                       time=duration_table_jb[i][j])

# 모든 경로 찾기
def find_all_routes(G, start, target, visited=None, path=None):
    if visited is None:
        visited = set()
    if path is None:
        path = []

    visited.add(start)
    path.append(start)

    if start == target:
        yield path.copy()
    else:
        for neighbor in G.neighbors(start):
            if neighbor not in visited:
                yield from find_all_routes(G, neighbor, target, visited, path)

    path.pop()
    visited.remove(start)

# 최적 경로 탐색 함수
def find_optimal_routes_within_range(G, stations, min_factor, max_factor, step):
    optimal_routes = []
    max_stops = 0
    explored_routes = set()  # 탐색된 경로를 저장할 집합

    # 여러 배수 값을 탐색
    for factor in np.arange(min_factor, max_factor + step, step):

        for station in stations.keys():
            if station != "전라북도 무주군":  # 목적지가 아니면
                routes = list(find_all_routes(G, station, "전라북도 무주군"))

                for route in routes:
                    valid_route = True
                    total_distance = 0
                    total_time = 0

                    # 각 경로의 모든 구간에 대해 확인
                    for i in range(len(route) - 1):
                        current_station = route[i]

                        # 현재 노드에서 무주군으로 바로 이동할 때의 거리와 시간
                        if "전라북도 무주군" in G[current_station]:  # 직접 경로가 있는 경우에만
                            direct_distance = G[current_station]["전라북도 무주군"]['distance']
                            direct_time = G[current_station]["전라북도 무주군"]['time']

                            # 현재 구간의 거리와 시간을 누적
                            total_distance += G[route[i]][route[i + 1]]['distance']
                            total_time += G[route[i]][route[i + 1]]['time']

                            # 각 중간 노드에서도 주어진 배수(factor) 조건 확인
                            if total_distance > direct_distance * factor or total_time > direct_time * factor:
                                valid_route = False
                                break

                    if valid_route:
                        # 경로를 집합에 저장할 수 있는 형태로 변환 (정렬하여 순서 상관없이 비교 가능)
                        route_tuple = tuple(sorted(route))

                        # 만약 해당 경로가 이전 배수에서 이미 탐색되었다면 제외
                        if route_tuple not in explored_routes:
                            explored_routes.add(route_tuple)  # 새로운 경로로 저장
                            # 가장 많은 정거장 수 확인
                            if len(route) > max_stops:
                                max_stops = len(route)
                                optimal_routes = [(route, total_distance, total_time, factor)]
                            elif len(route) == max_stops:
                                optimal_routes.append((route, total_distance, total_time, factor))

    return optimal_routes

# 탐색 범위 설정 (예: 1.1부터 1.5까지 0.1 간격으로 탐색)
min_factor = 1.1
max_factor = 1.5
step = 0.1

# 최적 경로 탐색 실행
optimal_routes = find_optimal_routes_within_range(G, stations, min_factor, max_factor, step)

# 결과 출력
print("경북지역 최적의 셔틀 노선")
if optimal_routes:
    for route, total_distance, total_time, factor in optimal_routes:
        print(f"최적의 셔틀 노선: {route} | 총 거리: {total_distance} | 총 시간: {total_time} | 배수: {factor:.2f}")
else:
    print("유효한 경로가 없습니다.")

경북지역 최적의 셔틀 노선
최적의 셔틀 노선: ['전라북도 전주시 완산구', '전라북도 전주시 덕진구', '전라북도 무주군'] | 총 거리: 179673.10169149653 | 총 시간: 148.697687989459 | 배수: 1.30
최적의 셔틀 노선: ['전라북도 군산시', '전라북도 익산시', '전라북도 무주군'] | 총 거리: 314301.7893354987 | 총 시간: 197.8089824642809 | 배수: 1.50
최적의 셔틀 노선: ['전라북도 전주시 덕진구', '전라북도 익산시', '전라북도 무주군'] | 총 거리: 306498.7115499178 | 총 시간: 198.47746238799544 | 배수: 1.50


### 전체지역 최적 셔틀 노선 (메모리부족으로 로컬환경 실행 불가)

In [6]:
shuttle_city_lst = [ '전라북도 전주시 덕진구', '대전광역시 서구', '전라북도 전주시 완산구',
                   '대전광역시 유성구', '대전광역시 동구', '전라북도 영동군', '세종특별자치시',
                   '충청남도 금산군', '대전광역시 중구', '전라북도 익산시','전라북도 진안군',
                   '경상남도 거창군', '대구광역시 달서구', '전라북도 군산시', '경상북도 김천시',
                   '대전광역시 대덕구', '경상남도 진주시', '전라북도 무주군']

In [7]:
duration_table = []
distance_table = []

for idx,target in enumerate(shuttle_city_lst):
    origin_criteria = list(set(address[address['시도 시군구'] == target]['행정동코드']))
    
    duration_arr = []
    distance_arr = []
    
    for idy, other in enumerate(shuttle_city_lst):
        if idx == idy:
            od_duration = od_dist = 0
        else:
            dest_criteria = list(set(address[address['시도 시군구'] == other]['행정동코드']))
            tmp_df = od_df[
                od_df['origin_hdong_cd'].isin(origin_criteria) & od_df['dest_hdong_cd'].isin(dest_criteria)
            ]
            od_duration = tmp_df['od_duration_avg'].mean()
            od_dist = tmp_df['od_dist_avg'].mean()
        duration_arr.append(od_duration)
        distance_arr.append(od_dist)

    duration_table.append(duration_arr)
    distance_table.append(distance_arr)

In [8]:
import numpy as np
import networkx as nx

# 정거장 후보지와 가중치
stations = {}

# 사람들이 많이 오는 top7 도시들은 가중치 2로 설정
for i in range(7):
    stations[shuttle_city_lst[i]] = 2

# 무주군은 목적지로서 가중치 0 설정
stations[shuttle_city_lst[-1]] = 0

# 나머지도시들은 가중치 1로 설정
for i in range(7, len(shuttle_city_lst) - 1):
    stations[shuttle_city_lst[i]] = 1


# 그래프 생성
G = nx.DiGraph()

# 그래프에 엣지 추가
for i, from_station in enumerate(stations.keys()):
    for j, to_station in enumerate(stations.keys()):
        if i != j:
            G.add_edge(from_station, to_station, 
                       distance=distance_table[i][j], 
                       time=duration_table[i][j])

# 모든 경로 찾기
def find_all_routes(G, start, target, visited=None, path=None):
    if visited is None:
        visited = set()
    if path is None:
        path = []

    visited.add(start)
    path.append(start)

    if start == target:
        yield path.copy()
    else:
        for neighbor in G.neighbors(start):
            if neighbor not in visited:
                yield from find_all_routes(G, neighbor, target, visited, path)

    path.pop()
    visited.remove(start)

# 최적 경로 탐색 함수
def find_optimal_routes_within_range(G, stations, min_factor, max_factor, step):
    optimal_routes = []
    max_stops = 0
    explored_routes = set()  # 탐색된 경로를 저장할 집합

    # 여러 배수 값을 탐색
    for factor in np.arange(min_factor, max_factor + step, step):

        for station in stations.keys():
            if station != "전라북도 무주군":  # 목적지가 아니면
                routes = list(find_all_routes(G, station, "전라북도 무주군"))

                for route in routes:
                    valid_route = True
                    total_distance = 0
                    total_time = 0

                    # 각 경로의 모든 구간에 대해 확인
                    for i in range(len(route) - 1):
                        current_station = route[i]

                        # 현재 노드에서 무주군으로 바로 이동할 때의 거리와 시간
                        if "전라북도 무주군" in G[current_station]:  # 직접 경로가 있는 경우에만
                            direct_distance = G[current_station]["전라북도 무주군"]['distance']
                            direct_time = G[current_station]["전라북도 무주군"]['time']

                            # 현재 구간의 거리와 시간을 누적
                            total_distance += G[route[i]][route[i + 1]]['distance']
                            total_time += G[route[i]][route[i + 1]]['time']

                            # 각 중간 노드에서도 주어진 배수(factor) 조건 확인
                            if total_distance > direct_distance * factor or total_time > direct_time * factor:
                                valid_route = False
                                break

                    if valid_route:
                        # 경로를 집합에 저장할 수 있는 형태로 변환 (정렬하여 순서 상관없이 비교 가능)
                        route_tuple = tuple(sorted(route))

                        # 만약 해당 경로가 이전 배수에서 이미 탐색되었다면 제외
                        if route_tuple not in explored_routes:
                            explored_routes.add(route_tuple)  # 새로운 경로로 저장
                            # 가장 많은 정거장 수 확인
                            if len(route) > max_stops:
                                max_stops = len(route)
                                optimal_routes = [(route, total_distance, total_time, factor)]
                            elif len(route) == max_stops:
                                optimal_routes.append((route, total_distance, total_time, factor))

    return optimal_routes

# 탐색 범위 설정 (예: 1.1부터 1.5까지 0.1 간격으로 탐색)
min_factor = 1.1
max_factor = 1.5
step = 0.1

# 최적 경로 탐색 실행
optimal_routes = find_optimal_routes_within_range(G, stations, min_factor, max_factor, step)

# 결과 출력
if optimal_routes:
    for route, total_distance, total_time, factor in optimal_routes:
        print(f"최적의 셔틀 노선: {route} | 총 거리: {total_distance} | 총 시간: {total_time} | 배수: {factor:.2f}")
else:
    print("유효한 경로가 없습니다.")


MemoryError: 

In [9]:
import numpy as np
import networkx as nx

# 정거장 후보지와 가중치
stations = {}

# 사람들이 많이 오는 top7 도시들은 가중치 2로 설정
for i in range(7):
    stations[shuttle_city_lst[i]] = 2

# 무주군은 목적지로서 가중치 0 설정
stations[shuttle_city_lst[-1]] = 0

# 나머지도시들은 가중치 1로 설정
for i in range(7, len(shuttle_city_lst) - 1):
    stations[shuttle_city_lst[i]] = 1


# 그래프 생성
G = nx.DiGraph()

# 그래프에 엣지 추가
for i, from_station in enumerate(stations.keys()):
    for j, to_station in enumerate(stations.keys()):
        if i != j:
            G.add_edge(from_station, to_station, 
                       distance=distance_table[i][j], 
                       time=duration_table[i][j])

def find_optimal_routes_within_range(G, stations, min_factor, max_factor, step):
    optimal_routes = []
    max_stops = 0

    for factor in np.arange(min_factor, max_factor + step, step):
        print(f"배수: {factor:.2f}")
        for station in stations.keys():
            if station != "전라북도 무주군":  # 목적지가 아니면
                for route in find_all_routes(G, station, "전라북도 무주군"):
                    valid_route = True

                    for i in range(len(route) - 1):
                        # 각 정거장에서 무주군으로 바로 갔을 때의 거리 및 시간
                        if "전라북도 무주군" in G[route[i]]:
                            direct_distance = G[route[i]]["전라북도 무주군"]['distance']
                            direct_time = G[route[i]]["전라북도 무주군"]['time']

                            total_distance = sum(G[route[i]][route[i + 1]]['distance'] for i in range(len(route) - 1))
                            total_time = sum(G[route[i]][route[i + 1]]['time'] for i in range(len(route) - 1))

                            # 배수 조건 확인
                            if total_distance > direct_distance * factor or total_time > direct_time * factor:
                                valid_route = False
                                break
                    
                    if valid_route:
                        if len(route) > max_stops:
                            max_stops = len(route)
                            optimal_routes = [(route, total_distance, total_time, factor)]
                        elif len(route) == max_stops:
                            optimal_routes.append((route, total_distance, total_time, factor))

    return optimal_routes

# 탐색 범위 설정 (예: 1.1부터 1.5까지 0.1 간격으로 탐색)
min_factor = 1.1
max_factor = 1.5
step = 0.1

# 최적 경로 탐색 실행
optimal_routes = find_optimal_routes_within_range(G, stations, min_factor, max_factor, step)

# 결과 출력
if optimal_routes:
    for route, total_distance, total_time, factor in optimal_routes:
        print(f"최적의 셔틀 노선: {route} | 총 거리: {total_distance} | 총 시간: {total_time} | 배수: {factor:.2f}")
else:
    print("유효한 경로가 없습니다.")


배수: 1.10


KeyboardInterrupt: 